Testando aqui para ver se consigo pegar os dados dentro do google colab

In [5]:
!git clone https://github.com/tiuwill/cnn-to-brain-cancer.git

Cloning into 'cnn-to-brain-cancer'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (239/239), done.
remote: Total 243 (delta 3), reused 240 (delta 3), pack-reused 0
Receiving objects: 100% (243/243), 6.84 MiB | 25.48 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [6]:
!ls cnn-to-brain-cancer

 data  'Detect Brain Cancer With CNN.ipynb'   README.md


In [0]:
import numpy as np
from glob import glob


def load_data(path, target):
  brains = glob(path)
  targets = [target] * len(brains)
  assert len(brains) == len(targets)
  return brains, targets

positive_brains, positive_targets = load_data('./cnn-to-brain-cancer/data/yes/*',1)
negative_brains, negative_targets = load_data('./cnn-to-brain-cancer/data/no/*', 0)

  

In [0]:
#JOININ DATA
import pandas as pd

brains = positive_brains + negative_brains

assert len(brains) == (len(positive_brains) + len(negative_brains))

targets = positive_targets + negative_targets

assert len(targets) == (len(positive_targets) + len(negative_targets))

data = pd.DataFrame()



In [0]:
#SPLITING TRAIN TEST AND VALIDATION

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(brains, targets, test_size=0.2, random_state=1, stratify=targets)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)


In [0]:
#PRocessando as imagens, dimensionando as imagens para que as mesmas tenham
#O mesmo tamanho e carregando as em um vetor

from keras.preprocessing import image
from tqdm import tqdm


def path_to_tensor(img_path):
  #Carregar a imagem em formato RGB
  img = image.load_img(img_path, target_size=(224, 224))
  #Convertendo para um array de 224, 224, 3
  x = image.img_to_array(img)
  
  #retorna um array de 4 dimensões sendo:
  #quantidade de imagens
  #altura
  #largura
  #rgb (3 camadas)
  return np.expand_dims(x, axis=0)
  
  
  
def paths_to_tensor(img_paths):
  list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
  return np.vstack(list_of_tensors)



In [24]:
#Pre processando as imagens
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True  

train_tensors = paths_to_tensor(X_train).astype('float32')/255
valid_tensors = paths_to_tensor(X_val).astype('float32')/255
test_tensors = paths_to_tensor(X_test).astype('float32')/255





100%|██████████| 51/51 [00:00<00:00, 307.96it/s]


In [30]:
#Criando uma rede neural para prever os dados de tumor no cerebro from scratch

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224, 224,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.4))
model.add(Dense(2, activation='sigmoid'))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 28, 28, 64)        0         
__________

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
!mkdir saved_models

In [95]:
#Training from scratch

from keras.callbacks import ModelCheckpoint  
from keras.utils import to_categorical


epochs = 50


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch_no_transfer.hdf5',
                              verbose=1, save_best_only=True)


model.fit(train_tensors, to_categorical(y_train),
         validation_data=(valid_tensors, to_categorical(y_val)),
         epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)




Train on 161 samples, validate on 41 samples
Epoch 1/50
161/161 [==============================] - 5s 34ms/step - loss: 0.5731 - acc: 0.7267 - val_loss: 0.6515 - val_acc: 0.6829

Epoch 00001: val_loss improved from inf to 0.65148, saving model to saved_models/weights.best.from_scratch_no_transfer.hdf5
Epoch 2/50
161/161 [==============================] - 5s 34ms/step - loss: 0.5637 - acc: 0.7081 - val_loss: 0.6682 - val_acc: 0.7073

Epoch 00002: val_loss did not improve from 0.65148
Epoch 3/50
161/161 [==============================] - 5s 34ms/step - loss: 0.5895 - acc: 0.7143 - val_loss: 0.6323 - val_acc: 0.7317

Epoch 00003: val_loss improved from 0.65148 to 0.63231, saving model to saved_models/weights.best.from_scratch_no_transfer.hdf5
Epoch 4/50
161/161 [==============================] - 5s 34ms/step - loss: 0.5545 - acc: 0.7391 - val_loss: 0.6523 - val_acc: 0.7317

Epoch 00004: val_loss did not improve from 0.63231
Epoch 5/50
161/161 [==============================] - 5s 34ms/ste

In [0]:
model.load_weights('saved_models/weights.best.from_scratch_no_transfer.hdf5')

In [0]:
from google.colab import files
files.download('saved_models/weights.best.from_scratch_no_transfer.hdf5') 

In [97]:
# get index of predicted dog breed for each image in test set
predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]


# report test accuracy
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(to_categorical(y_test), axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 72.5490%


In [67]:
print(to_categorical(y_test))

[[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [68]:
print(predictions)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0]


In [0]:
results = np.array(predictions)==y_test

In [0]:
results2 = np.array(predictions)==np.argmax(to_categorical(y_test), axis=1)

In [0]:
assert results.all() == results2.all()

In [80]:

#transfer learning com resnet
from keras import applications 


vgg_model = applications.VGG16(weights='imagenet', 
                      include_top=False, 
                      input_shape=(224, 224, 3))

vgg_model.summary()

58892288/58889256 [==============================] - 1s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool

In [0]:
layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])


In [0]:
x = layer_dict['block5_pool'].output
x = GlobalAveragePooling2D(input_shape=train_tensors.shape[1:])(x)
x = Dense(2, activation='sigmoid')(x)

In [90]:
from keras.models import Model

transfer_model = Model(input=vgg_model.input, output=x)

#previnindo para não treinar a rede nos nós já treinados
for layer in transfer_model.layers[:18]:
  layer.trainable = False
  
transfer_model.summary()

transfer_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [92]:

#Treinando o novo modelo
from keras.callbacks import ModelCheckpoint  
from keras.utils import to_categorical


epochs = 50


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5',
                              verbose=1, save_best_only=True)


transfer_model.fit(train_tensors, to_categorical(y_train),
         validation_data=(valid_tensors, to_categorical(y_val)),
         epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 161 samples, validate on 41 samples
Epoch 1/50
161/161 [==============================] - 103s 642ms/step - loss: 0.8395 - acc: 0.3851 - val_loss: 0.7335 - val_acc: 0.3902

Epoch 00001: val_loss improved from inf to 0.73351, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/50
161/161 [==============================] - 101s 625ms/step - loss: 0.6946 - acc: 0.5217 - val_loss: 0.6453 - val_acc: 0.6098

Epoch 00002: val_loss improved from 0.73351 to 0.64528, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 3/50
161/161 [==============================] - 101s 626ms/step - loss: 0.6495 - acc: 0.6149 - val_loss: 0.6369 - val_acc: 0.6098

Epoch 00003: val_loss improved from 0.64528 to 0.63692, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 4/50
161/161 [==============================] - 101s 625ms/step - loss: 0.6418 - acc: 0.6149 - val_loss: 0.6333 - val_acc: 0.6098

Epoch 00004: val_loss improved from 0.63692 to 0.63329, saving 

In [93]:
# get index of predicted dog breed for each image in test set
predictions = [np.argmax(transfer_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]


# report test accuracy
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(to_categorical(y_test), axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 82.3529%


In [0]:
from google.colab import files
files.download('saved_models/weights.best.from_scratch.hdf5') 